<center><img src="car.jpg" width=500></center>


Insurance companies invest a lot of time and money into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries insurance it is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

(`Source: https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf`) 

Knowing all of this, On the Road car insurance have requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to identify the single feature that results in the best performing model, as measured by accuracy, so they can start with a simple model in production.

They have supplied you with their customer data as a csv file called `car_insurance.csv`, along with a table detailing the column names and descriptions below.



## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-25</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code | 
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client | 
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

In [31]:
# Import required modules
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit

# Start coding!

In [32]:
df = pd.read_csv('car_insurance.csv')

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10000 non-null  int64  
 1   age                  10000 non-null  int64  
 2   gender               10000 non-null  int64  
 3   driving_experience   10000 non-null  object 
 4   education            10000 non-null  object 
 5   income               10000 non-null  object 
 6   credit_score         9018 non-null   float64
 7   vehicle_ownership    10000 non-null  float64
 8   vehicle_year         10000 non-null  object 
 9   married              10000 non-null  float64
 10  children             10000 non-null  float64
 11  postal_code          10000 non-null  int64  
 12  annual_mileage       9043 non-null   float64
 13  vehicle_type         10000 non-null  object 
 14  speeding_violations  10000 non-null  int64  
 15  duis                 10000 non-null  

In [33]:
#lets use describe so we can figure out some candidate features.
print(df.describe())

                  id           age  ...  past_accidents       outcome
count   10000.000000  10000.000000  ...    10000.000000  10000.000000
mean   500521.906800      1.489500  ...        1.056300      0.313300
std    290030.768758      1.025278  ...        1.652454      0.463858
min       101.000000      0.000000  ...        0.000000      0.000000
25%    249638.500000      1.000000  ...        0.000000      0.000000
50%    501777.000000      1.000000  ...        0.000000      0.000000
75%    753974.500000      2.000000  ...        2.000000      1.000000
max    999976.000000      3.000000  ...       15.000000      1.000000

[8 rows x 13 columns]


In [34]:
#some clear missing values, lets impute the since they are roughly 10% of the data

#for credit card, our imputing strategy will be grouping them by income then getting mean credit score for the filler.
df['credit_score'] = df['credit_score'].fillna(
df.groupby('income')['credit_score'].transform('mean')
)

#annual mileage will be grouped by driving experience
df['annual_mileage'] = df['annual_mileage'].fillna(
    df.groupby('driving_experience')['annual_mileage'].transform('mean')
)

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10000 non-null  int64  
 1   age                  10000 non-null  int64  
 2   gender               10000 non-null  int64  
 3   driving_experience   10000 non-null  object 
 4   education            10000 non-null  object 
 5   income               10000 non-null  object 
 6   credit_score         10000 non-null  float64
 7   vehicle_ownership    10000 non-null  float64
 8   vehicle_year         10000 non-null  object 
 9   married              10000 non-null  float64
 10  children             10000 non-null  float64
 11  postal_code          10000 non-null  int64  
 12  annual_mileage       10000 non-null  float64
 13  vehicle_type         10000 non-null  object 
 14  speeding_violations  10000 non-null  int64  
 15  duis                 10000 non-null  

In [35]:
#next cleaning step is to convert all oibjects  into numeric codes

for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].astype('category').cat.codes

print(df.head())

       id  age  gender  ...  duis  past_accidents  outcome
0  569520    3       0  ...     0               0      0.0
1  750365    0       1  ...     0               0      1.0
2  199901    0       0  ...     0               0      0.0
3  478866    0       1  ...     0               0      0.0
4  731664    1       1  ...     0               1      1.0

[5 rows x 18 columns]


In [36]:
#create a list of models
features = df.columns.drop(['id', 'outcome'])
best_feature = ""
best_accuracy = 0
model_results = []

In [37]:
for col in features:
    formula = f"outcome ~ {col}"

    #fit model
    model = logit(formula, data = df).fit(disp = 0)
    pred_probs = model.predict(df)
    predictions = np.where(pred_probs > 0.5, 1, 0)
    accuracy = (predictions == df['outcome']).mean()
    model_results.append({'feature': col, 'accuracy': accuracy, 'model_obj': model})
    print(f"Feature: {col:<20} Accuracy: {accuracy:.4f}")
    #replace best performer
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_feature = col
    

Feature: age                  Accuracy: 0.7747
Feature: gender               Accuracy: 0.6867
Feature: driving_experience   Accuracy: 0.7771
Feature: education            Accuracy: 0.6867
Feature: income               Accuracy: 0.6867
Feature: credit_score         Accuracy: 0.7116
Feature: vehicle_ownership    Accuracy: 0.7351
Feature: vehicle_year         Accuracy: 0.6867
Feature: married              Accuracy: 0.6867
Feature: children             Accuracy: 0.6867
Feature: postal_code          Accuracy: 0.6867
Feature: annual_mileage       Accuracy: 0.6920
Feature: vehicle_type         Accuracy: 0.6867
Feature: speeding_violations  Accuracy: 0.6867
Feature: duis                 Accuracy: 0.6867
Feature: past_accidents       Accuracy: 0.6867


In [38]:
print(f"Best Feature: '{best_feature}' with {best_accuracy:.2%} accuracy")

Best Feature: 'driving_experience' with 77.71% accuracy


In [39]:
#set desired output for work
best_feature_df = pd.DataFrame({"best_feature": [best_feature], "best_accuracy": [best_accuracy]})

print(best_feature_df)

         best_feature  best_accuracy
0  driving_experience         0.7771
